In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from unyt import yr, Myr, kpc, arcsec, nJy, Mpc, Msun, erg, s, Hz, kelvin
%pip install schwimmbad
import schwimmbad
import h5py

from synthesizer.grid import Grid
from synthesizer.load_data.load_camels import load_CAMELS_SwiftEAGLE_subfind
from synthesizer.sed import Sed
from synthesizer.filters import Filter, FilterCollection
from synthesizer.sed import combine_list_of_seds
from synthesizer.emission_models import (
    IncidentEmission,
)

from synthesizer.conversions import lnu_to_absolute_mag, fnu_to_apparent_mag, fnu_to_lnu

Note: you may need to restart the kernel to use updated packages.


/tmp/ipykernel_5692/3905042009.py:12: FutureWarning: The filters module has been moved to the instruments module. Please update your imports synthesizer.filters -> synthesizer.instruments
  from synthesizer.filters import Filter, FilterCollection


In [2]:
# get grid for all galaxies
# gives grid of metalicity and age which is used to map on our camels galaxies
grid_name = "bc03_chabrier03-0.1,100.hdf5"
#grid_name = "bc03-2016-Miles_chabrier-0.1,100.hdf5" # try old grid, LF looks weird?!
grid_dir = "/home/jovyan/"
grid = Grid(grid_name, grid_dir=grid_dir, read_lines=False)

In [3]:
# get gals

CV_X = 'CV_18'
_dir = '/home/jovyan/Data/Sims/Swift-EAGLE/L25n256/CV/' + CV_X
group_dir = '/home/jovyan/Data/FOF_Subfind/Swift-EAGLE/L25n256/CV/' + CV_X


In [ ]:

gals_074 = load_CAMELS_SwiftEAGLE_subfind(
    _dir='/home/jovyan/Data/Sims/Swift-EAGLE/L25n256/CV/' + CV_X,
    snap_name="snapshot_074.hdf5",
    group_name="groups_074.hdf5",
    group_dir='/home/jovyan/Data/FOF_Subfind/Swift-EAGLE/L25n256/CV/' + CV_X
)



In [ ]:

print('Number of Galaxies in snap_074', len(gals_074))

cat_074 = group_dir+'/groups_074.hdf5'
# open file
f = h5py.File(cat_074, 'r')

# read different attributes of the header
boxSize_074 = f['Header'].attrs[u'BoxSize']/1e3 #Mpc/h
redshift_074 = f['Header'].attrs[u'Redshift']
print('BoxSize: ', boxSize_074)
print('redshift: ', redshift_074)

In [ ]:
import h5py

# Open the HDF5 file for snapshot 074
cat_074 = group_dir + '/groups_074.hdf5'
f = h5py.File(cat_074, 'r')

# Read different attributes of the header
boxSize_074 = f['Header'].attrs[u'BoxSize']/1e3  # Mpc/h
redshift_074 = f['Header'].attrs[u'Redshift']    # You said this seems wrong
scale_factor_074 = 0.681#f['Header'].attrs[u'Scale-factor']  # Let's grab the scale factor

# Compute the redshift from the scale factor
redshift_computed_074 = (1 / scale_factor_074) - 1

# Print the information
print('BoxSize: ', boxSize_074)
print('Redshift (from file): ', redshift_074)
print('Scale factor: ', scale_factor_074)
print('Computed Redshift (from scale factor): ', redshift_computed_074)


In [ ]:
gal =gals_074[0]
gal.gas.nparticles

In [ ]:
# Filter galaxies to only include those with 100 or more star particles
## NEED TO ADD FILTER TO GET RID OF ANY GALAXIES WITH LESS THAN 100 star Particles!! This is the resolution limit!!
# Chris: when you load your galaxies in could you do a filter for those with at least 100 star particles? that's the resolution limit

gals_074 = [gal for gal in gals_074 if len(gal.stars.ages) >= 100]  # this shouldnt be here?! donesnt make sense
gals_074 = [gal for gal in gals_074 if gal.stars.nstars >= 100]
#gals_074 = [gal for gal in gals_074 if gal.gas.nparticles >= 100]

# Ensure that the gas component exists and has 100 or more particles
gals_074 = [gal for gal in gals_074 if gal.gas is not None and hasattr(gal.gas, 'nparticles') and gal.gas.nparticles >= 100]

In [ ]:
incident = IncidentEmission(grid)
little_h =  0.6711

spec_list = []
# Lets work with z=0 so gals_025
for i in gals_074:
    gal = i
    # get_spectra_incident An Sed object containing the stellar spectra
    spec = gal.stars.get_spectra(incident)
    spec.get_fnu0()
    spec_list.append(spec)


In [ ]:
# combine
seds = combine_list_of_seds(spec_list)
seds

In [ ]:
seds.lnu  # get luminosity
# A typical value for the spectral luminosity density of a galaxy is around 28 erg/(Hz*s)

In [ ]:
# Need to use the grid lam, dont need to define wavelengths here: see codes here: http://svo2.cab.inta-csic.es/theory/fps/index.php?id=Swift/UVOT.UVM2&&mode=browse&gname=Swift&gname2=UVOT#filter
# need to use transmission NOT area
# Get for XMM
fil_uvm2_XMM = Filter("XMM/OM.UVM2_filter", new_lam=grid.lam)

# now get for UVOT
fil_uvm2_UVOT = Filter("Swift/UVOT.UVM2_fil", new_lam=grid.lam) # changed from new_lam=lams_uvot to grid.lam and get 0 transmission, but warning sign is back

# what is a top hat filter?
filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)

# 0 = TopHat, 1 = XMM, 2= Swift
filt_lst = [filt1, fil_uvm2_XMM, fil_uvm2_UVOT]

# get filters in combined array to apply to all galaxies
combined_filters = FilterCollection(
    filters=filt_lst, new_lam=grid.lam
)


In [ ]:
combined_filters.filters

In [ ]:
seds.photo_lnu

In [ ]:
phot = seds.get_photo_lnu(combined_filters, verbose=True) 
print(phot)
# phot_mags = [lnu_to_absolute_mag(spec.photo_lnu[f] for f in combined_filters]

In [ ]:
abs_mags = [lnu_to_absolute_mag(phot[f]) for f in combined_filters.filters]
#print(abs_mags)


# check 
from astropy.cosmology import Planck18    # You can choose other cosmologies if needed

def apparent_magnitude_from_absolute(redshift, absolute_magnitude):
    # Get the luminosity distance for the given redshift
    # Distnaces are in parsecs
    d_L_pc = Planck18.luminosity_distance(redshift).to('pc').value
    
    # Calculate the apparent magnitude using the distance modulus formula
    apparent_mag = absolute_magnitude - 5 + (5 * np.log10(d_L_pc))
    
    return apparent_mag

ap_test = apparent_magnitude_from_absolute(redshift_074, abs_mags)
ap_test[1] # 1 is XMM-OM


Absolute magnitude is defined to be the apparent magnitude an object would have if it were located at a distance of 10 parsecs.
In astronomy, absolute magnitude (M) is a measure of the luminosity of a celestial object on an inverse logarithmic astronomical magnitude scale.

In [ ]:
abs_mag_th = abs_mags[0]

In [ ]:
# next steps, get luminosity function for these magnitudes
# co-moving volume: BoxSize_025 and redshift:
little_h =  0.6711
Vphys = (boxSize_074/little_h )**3
Vcom = Vphys * ((1+redshift_074)**3)
print(Vcom)


In [ ]:
# Alternative method for LF:
# try this method again, but using AB mag instead of mass, and suply your own bins (up to -17, say)
def calc_df(ab_mag, volume, massBinLimits):

# OG:        hist, dummy = np.histogram(np.log10(mstar), bins = massBinLimits)
        hist, dummy = np.histogram(ab_mag, bins = massBinLimits)
        hist = np.float64(hist)
        phi = (hist / volume) / (massBinLimits[1] - massBinLimits[0])

        phi_sigma = (np.sqrt(hist) / volume) /\
                    (massBinLimits[1] - massBinLimits[0]) # Poisson errors

        return phi, phi_sigma, hist

In [ ]:
# Define the bin edges for AB magnitudes
# Here, let's define bins from -25 to -17 in steps of 0.5 magnitude
massBinLimits = np.arange(-22, -16, 0.5)

phi, phi_sigma, hist = calc_df(abs_mag_th, Vcom, massBinLimits)

In [ ]:

label_025 = 'z = ', redshift_074
label_th = 'Tophat'

# Plot the luminosity function
plt.errorbar(massBinLimits[:-1], phi, yerr=phi_sigma, fmt='o', color='blue', label=label_th)

plt.xlabel('Absolute Magnitude (AB)')
plt.ylabel('Number Density (Mpc^-3 mag^-1)')
plt.yscale('log')
plt.legend(loc='lower right')


plt.title('Luminosity Function Top Hat filter')
plt.grid(True)
plt.show()